In [ ]:
# conda env list

# conda create  -n [이름] python=[버전]
# conda activate [이름]

# conda deactivate
# conda remove -n [이름] --all

In [ ]:
# 입력-> 토큰화 및 시퀀스 변화 -> 패딩(고정길이화) -> 임베딩(단어->벡터화)-> 1D Convolution + poling 반복
# -> Flatten ->Dense(은닉) -> 출력(소프트맥스, 이진분류) -> 학습(Adam + binary_Crossentropy)
# -> 검증/테스트 평가 -> 시각화

- 말뭉치 로딩(nltk)  데이터 로딩
- 토큰화(빈고 기반 인덱싱) 텍스트를 숫자로 변환
- 시퀀스 패딩   고정길이 배치 구성
- 임베딩  단어를 dense vector표현 학습
- 임베딩  발전
    - 한계 : 작은데이터에서는 일반화 부족
    - 발전 : 사전학습(Word2vec), 문맥적 임베딩(BERT, GPT)

In [11]:
# 토큰화
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
# sample data
texts = [
    'I really love this movie',
    'I hate this boring film',
    'love love great film'
]
# 토큰화 객체(최대 단어 10, oov 토큰 지정)
tokenizer = Tokenizer(num_words=10, oov_token='UNK')
tokenizer.fit_on_texts(texts)
print(f'단어인덱스 : {tokenizer.word_index}')
# 시퀀스
seqs = tokenizer.texts_to_sequences(texts)
print(f'원본 시퀀스 : {seqs}')
# 패딩(최대 길이를 6)
padded = pad_sequences(seqs, maxlen=6,padding='post')
print(f'패딩결과 : {padded}  사이즈 : {padded.shape}')


단어인덱스 : {'UNK': 1, 'love': 2, 'i': 3, 'this': 4, 'film': 5, 'really': 6, 'movie': 7, 'hate': 8, 'boring': 9, 'great': 10}
원본 시퀀스 : [[3, 6, 2, 4, 7], [3, 8, 4, 9, 5], [2, 2, 1, 5]]
패딩결과 : [[3 6 2 4 7 0]
 [3 8 4 9 5 0]
 [2 2 1 5 0 0]]  사이즈 : (3, 6)


In [13]:
# 임베딩 : 임베딩 레이어
import tensorflow as tf
# 패딩된 시퀀스  padded
vocab_size = 11  # unk 포함 단어인덱스 최대값 + 1
embed_dim = 4  # 작은 차원
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=vocab_size,output_dim=embed_dim, input_length=6)    
])
embeddings = model.predict(padded)
print(f'임베딩 텐서 모양 : {embeddings.shape}')  #(3,6,4)
print(f'첫 문장 첫 단어 벡터 : {embeddings[0,0,:]}')

c:\Users\playdata2\miniconda3\envs\LLM311\Lib\site-packages\keras\src\layers\core\embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 242ms/step
임베딩 텐서 모양 : (3, 6, 4)
첫 문장 첫 단어 벡터 : [ 0.04000187 -0.02458723  0.01764833  0.0096667 ]


In [17]:
# 1D Convolution
import numpy as np
import tensorflow as tf
# 임의 시퀀스(배치=1, 길이=6, 임베딩=4)
x = np.random.randn(1,6,4).astype('float32')
conv = tf.keras.layers.Conv1D(
    filters = 2  # 2개의 패턴을 감지  긍정 부정을 감지
    ,kernel_size=3  # 3-gram
    ,activation='relu'  
)
y = conv(x)
print(f'입력 shape {x.shape}')
print(f'출력 shape {y.shape}')
print(f'출력값  {y.numpy()}')

입력 shape (1, 6, 4)
출력 shape (1, 4, 2)
출력값  [[[0.21842088 1.464069  ]
  [0.         0.08337156]
  [0.         0.        ]
  [2.0848384  1.3942744 ]]]


In [18]:
# MaxPooling
pool = tf.keras.layers.MaxPooling1D(pool_size=2)
pooled = pool(y)
print(f'before pooling :{y.shape}')
print(f'after pooling :{pooled.shape}')

before pooling :(1, 4, 2)
after pooling :(1, 2, 2)


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer  # 단어를 단어사전, 단어를 숫자로 변경
from tensorflow.keras.preprocessing.sequence import pad_sequences  #길이 맞추기
texts = [
    'I really love this movie',  # 긍정
    'I hate this boring film',  # 부정
    'great love movie',  # 긍정
    'boring hate film' # 부정
]
labels = np.array([0,1,0,1])

tokenizer = Tokenizer(num_words=50,oov_token='UNK')
tokenizer.fit_on_texts(texts)  # 단어사전 생성
seqs =  tokenizer.fit_on_sequences(texts)  # 단어사전기반으로 단어들을 숫자로 변경(길이는 다름)
x = pad_sequences(seqs,maxlen=6,padding='post')
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(51,8,input_length=6),
    tf.keras.layers.Conv1D(16,3,activation='relu'),
    tf.keras.layers.GlobalAveragePooling1D(),
])
